In [92]:
import numpy as np 
import pandas as pd
import matplotlib .pyplot as plt 
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import datetime
import operator
import math
import random
from scipy.linalg import norm
from scipy.spatial.distance import cdist
from collections import defaultdict
from sklearn import datasets
from sklearn.datasets import make_blobs
import warnings
import os, cv2, json

In [93]:
data = pd.read_csv('./Country-data.csv')

In [94]:
data.shape

(167, 10)

In [95]:
data.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [96]:
columns = data.columns
columns

Index(['country', 'child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp'],
      dtype='object')

In [97]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['country'])
le.transform(data['country'])
data['country'] = le.transform(data['country'])
print(data.shape)
data.head()

(167, 10)


,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,0,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,1,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,2,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,3,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,4,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [98]:
scalarModel = StandardScaler()
data = scalarModel.fit_transform(data)
data = pd.DataFrame(data , columns = columns )
data

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,-1.721710,1.291532,-1.138280,0.279088,-0.082455,-0.808245,0.157336,-1.619092,1.902882,-0.679180
1,-1.700967,-0.538949,-0.479658,-0.097016,0.070837,-0.375369,-0.312347,0.647866,-0.859973,-0.485623
2,-1.680223,-0.272833,-0.099122,-0.966073,-0.641762,-0.220844,0.789274,0.670423,-0.038404,-0.465376
3,-1.659480,2.007808,0.775381,-1.448071,-0.165315,-0.585043,1.387054,-1.179234,2.128151,-0.516268
4,-1.638736,-0.695634,0.160668,-0.286894,0.497568,0.101732,-0.601749,0.704258,-0.541946,-0.041817
...,...,...,...,...,...,...,...,...,...,...
162,1.638736,-0.225578,0.200917,-0.571711,0.240700,-0.738527,-0.489784,-0.852161,0.365754,-0.546913
163,1.659480,-0.526514,-0.461363,-0.695862,-1.213499,-0.033542,3.616865,0.546361,-0.316678,0.029323
164,1.680223,-0.372315,1.130305,0.008877,1.380030,-0.658404,0.409732,0.286958,-0.661206,-0.637754
165,1.700967,0.448417,-0.406478,-0.597272,-0.517472,-0.658924,1.500916,-0.344633,1.140944,-0.637754


In [99]:
data = data.values
data

array([[-1.72171011,  1.29153238, -1.13827979, ..., -1.61909203,
         1.90288227, -0.67917961],
       [-1.70096662, -0.5389489 , -0.47965843, ...,  0.64786643,
        -0.85997281, -0.48562324],
       [-1.68022312, -0.27283273, -0.09912164, ...,  0.67042323,
        -0.0384044 , -0.46537561],
       ...,
       [ 1.68022312, -0.37231541,  1.13030491, ...,  0.28695762,
        -0.66120626, -0.63775406],
       [ 1.70096662,  0.44841668, -0.40647827, ..., -0.34463279,
         1.14094382, -0.63775406],
       [ 1.72171011,  1.11495062, -0.15034774, ..., -2.09278484,
         1.6246091 , -0.62954556]])

In [100]:
def euclidean_distance(point1,point2):
    dis=0
    for i in range(len(point1)):
        dis+=(point1[i]-point2[i])**2
    return dis**0.5

In [101]:
g=2   
n=len(data)
c=3 #no. clusters
p=len(data[0])
max_iter=10
n,p

(167, 10)

In [102]:
def initialize_membership_matrix(n,c):
    member_mat=list()
    for i in range(n):
        random_list=[random.random() for x in range(c)]
        summation=sum(random_list)
        for i in range(len(random_list)):
            random_list[i]=random_list[i]/summation
        member_mat.append(random_list)
    return member_mat

def update_centroid(member_mat):
    centroids={}

    for j in range(c):
        temp=[]
        for k in range(p):
            
            add=0
            for i in range(n):
                add+=member_mat[i][j]**g
            x=0
            for i in range(n):
                x+=(member_mat[i][j]**g)*(data[i][k])
            val=x/add
            temp.append(val)
        centroids[j]=temp
    return centroids


def update_membership_matrix(member_mat,centroids):
    ratio=float(2/(g-1))

    for i in range(n):
        distances=list()
        for j in range(c):
            distances.append(euclidean_distance(data[i],centroids[j]))
        for j in range(c):
            den = sum([math.pow(float(distances[j]/distances[q]), ratio) for q in range(c)])
            member_mat[i][j] = float(1/den) 
           
            
    return member_mat

In [103]:
def find_cluster(member_mat):
    clusters=list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(member_mat[i]))
        clusters.append(idx)
    return clusters

def check(old_member_mat,member_mat):
    diff=0
    for i in range(n):
        for j in range(c):
            diff+=old_member_mat[i][j]-member_mat[i][j]               #old_member_mat---------> initialize_membership_matrix
    if(diff<0.01):
        return True
    return False

def fuzzy_c_mean():
    
    member_mat=initialize_membership_matrix(n,c)
    for i in range(max_iter):
        centroids=update_centroid(member_mat)
        old_member_mat=member_mat
        member_mat=update_membership_matrix(member_mat,centroids)
        cluster=find_cluster(member_mat)
        if(check(old_member_mat,member_mat))<0.01:
            print(i)
            break
    
   
    
    return cluster,centroids

In [104]:
clusters,centroids=fuzzy_c_mean()
print("Final Centroid points are:")
print(centroids)


# labeling the clusters
def label_clusters(clusters):
    z=0
    o=0
    t=0
    dict=defaultdict(int)


    for i in range(50):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    classes=[]
    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(50,100):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2

    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(100,150):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    fin1=max(z,max(o,t))
   
    classes.append(dict[fin1])
    
    return classes

classes=label_clusters(clusters)


Final Centroid points are:
{0: [0.005902254096222166, -0.3303360506551799, 0.08905172824524736, -0.02312108762861431, 0.06457600503891082, 0.05310980900695767, -0.06464591732018878, 0.2748988618536533, -0.3433710737313907, -0.0031979142733555926], 1: [-0.01023527748062941, 0.9694450565685299, -0.36128035442401174, -0.30347595696524826, -0.17565935651738618, -0.5662589358674243, 0.2022053993059101, -0.9195061861638271, 1.0164528951372385, -0.4975182298158407], 2: [-0.03802111371838655, -0.5172481764575808, 0.19264170973791775, 0.31617776878730836, 0.05140541305682131, 0.44268853766706023, -0.2227741463811996, 0.5595264954683907, -0.5281641864810261, 0.4646020706753556]}


In [105]:


def confusion_matrix(clusters,classes,labels):
    c_matrix=[[0 for i in range(labels)] for j in range(labels)]
    m=0
    n=0
    q=0
    for i in range(50):

        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[0][0]=m
    c_matrix[0][1]=n
    c_matrix[0][2]=q
    m=0
    n=0
    q=0
    for i in range(50,100):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[1][0]=m
    c_matrix[1][1]=n
    c_matrix[1][2]=q
    m=0
    n=0
    q=0

    for i in range(100,150):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[2][0]=m
    c_matrix[2][1]=n
    c_matrix[2][2]=q
    
    
    return c_matrix
    

In [106]:
class Metrics:
    
    def __init__(self,confusion_m):
        self.confusion_m=confusion_m
        self.total=np.sum(confusion_m)
        self.diagonal=np.sum(np.diag(confusion_m))
    
    def accuracy(self):
        accuracy=(self.diagonal/self.total)
        return accuracy
    
    def recall(self):
        recall=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=1)
        recall=np.mean(recall)
        return recall
    
    def precision(self):
        precision=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=0)
        precision=np.mean(precision)
        
        return precision
    
    def f1_score(self,precision,recall):
        f1_score=(2*precision*recall)/(precision+recall)
        
        return f1_score

In [107]:
matrix=confusion_matrix(clusters,classes,3)
performance=Metrics(matrix)

accuracy=performance.accuracy()
recall=performance.recall()
precision=performance.precision()
f1_score=performance.f1_score(precision,recall)

In [108]:
print('Making Calculations... ')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print("BEFORE USING PCA...")
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

A=initialize_membership_matrix(n,c)
print('initialize_membership_matrix: ',A ,end='\n')
print(' the_shape_of_initializing_membership_matrix:', len(A),end='\n')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

B=update_centroid(initialize_membership_matrix(n,c))
print('centroids are: ', B ,end='\n')       
print(' the_shape_of_centroids:', len(B),end='\n')

print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

C=update_membership_matrix(A,B)
print('update_membership_matrix: ',C ,end='\n')
print(' the_shape_of_updating_membership_matrix:', len(C),end='\n')

print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

D=find_cluster(C)
print('find_cluster: ',D ,end='\n')
print(' the_shape_of_clusters:', len(D),end='\n')

print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

print('confusion matrix is:',end='\n')
print(np.array(matrix),end='\n')
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')


print("Accuracy of the model is {}".format(accuracy*100))
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

print("Recall of the model is {}".format(recall*100))
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

print("Precision of the model is {}".format(precision*100))
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

print("F1-Score of the model is {}".format(f1_score*100))
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
data = pd.DataFrame(data , columns = columns )
data['cluster'] = D



Making Calculations... 
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BEFORE USING PCA...
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
initialize_membership_matrix:  [[0.523126774837559, 0.4696247127085631, 0.007248512453877809], [0.1428263367828399, 0.524137945697851, 0.33303571751930905], [0.38867403866270633, 0.0219049853390549, 0.5894209759982387], [0.5329717068387235, 0.4608988172988387, 0.006129475862437778], [0.23895954654181112, 0.06845274250105024, 0.6925877109571387], [0.3984686889659233, 0.5366856158870621, 0.06484569514701467], [0.4857921715297098, 0.4148430377645718, 0.09936479070571828], [0.2914061559597166, 0.2794965505632

In [110]:
print('FCM_Before_PCA with clusters')
data.head()

FCM_Before_PCA with clusters


,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,cluster
0,-1.721710,1.291532,-1.138280,0.279088,-0.082455,-0.808245,0.157336,-1.619092,1.902882,-0.679180,1
1,-1.700967,-0.538949,-0.479658,-0.097016,0.070837,-0.375369,-0.312347,0.647866,-0.859973,-0.485623,0
2,-1.680223,-0.272833,-0.099122,-0.966073,-0.641762,-0.220844,0.789274,0.670423,-0.038404,-0.465376,0
3,-1.659480,2.007808,0.775381,-1.448071,-0.165315,-0.585043,1.387054,-1.179234,2.128151,-0.516268,1
4,-1.638736,-0.695634,0.160668,-0.286894,0.497568,0.101732,-0.601749,0.704258,-0.541946,-0.041817,0
